## Summarization Pipeline
> In this pipeline we are using BART for summarizing transcript chunks over 1000 or 800 and refining it using LLM


In [ ]:
import os
import time
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
HF_TOKEN = os.getenv("HF_TOKEN")

In [ ]:
from langchain_groq.chat_models import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains.summarize import load_summarize_chain
from langchain_core.documents import Document
from huggingface_hub import InferenceClient
from typing import List

llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama-3.3-70b-versatile", # llama-3.3-70b-versatile good for summarizing texts and providing good context
    temperature=0.0,
    max_retries=2
)

hf_client = InferenceClient( # Using facebook/bart-large-cnn
    provider="hf-inference",
    api_key=HF_TOKEN
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

refine_template = PromptTemplate(template="""You are an expert meeting analyst. Your task is to create a comprehensive meeting summary.

Existing summary (if any):
{existing_answer}

New information from the meeting:
{text}

Instructions:
- Combine the existing summary with the new information
- Create a concise yet comprehensive summary
- Focus on key decisions, action items, and important discussion points
- Maintain a professional, clear tone
- Organize information logically

Refined Summary""")



In [ ]:
# Step 2: BART Summarization Function
def summarize_chunk_with_bart(chunk_text: str, max_length: int = 150, min_length: int = 50) -> str:
    """Summarize a single chunk using BART model via HuggingFace Inference."""
    try:
        summary = hf_client.summarization(
            chunk_text,
            model="facebook/bart-large-cnn",
            max_length=max_length,
            min_length=min_length
        )
        return summary.summary_text if hasattr(summary, 'summary_text') else summary
    except Exception as e:
        print(f"Error summarizing chunk: {e}")
        return chunk_text[:200]  # Fallback to truncated text

# Step 3: Process all chunks with BART
def summarize_chunks_with_bart(chunks: List[str]) -> List[str]:
    """Summarize all chunks using BART."""
    bart_summaries = []
    
    for i, chunk in enumerate(chunks):
        print(f"Summarizing chunk {i+1}/{len(chunks)} with BART...")
        summary = summarize_chunk_with_bart(chunk)
        bart_summaries.append(summary)
        time.sleep(0.6)  # Rate limiting
    
    return bart_summaries



In [ ]:
initial_template = """You are an expert meeting analyst. Summarize the following meeting content:

{text}

Create a concise, detailed summary focusing on:
- Key discussion points
- Decisions made
- Action items
- Important insights

Summary:"""

initial_prompt = PromptTemplate(
    template=initial_template,
    input_variables=["text"]
)

# Step 5: Main Pipeline Function
def summarize_meeting_transcript(transcript: str) -> dict:
    """
    Complete pipeline to summarize meeting transcript.
    
    Args:
        transcript: The full meeting transcript text
        
    Returns:
        dict containing:
            - final_summary: The comprehensive final summary
            - bart_summaries: Individual BART summaries of chunks
            - chunk_count: Number of chunks processed
    """
    print("Starting meeting transcript summarization pipeline...")
    
    # Split transcript into chunks
    print("\n[1/3] Splitting transcript into chunks...")
    chunks = text_splitter.split_text(transcript)
    print(f"Created {len(chunks)} chunks")
    
    # Summarize each chunk with BART
    print("\n[2/3] Summarizing chunks with BART...")
    bart_summaries = summarize_chunks_with_bart(chunks)
    
    # Combine BART summaries into Documents for LangChain
    print("\n[3/3] Combining summaries with LLM using refine chain...")
    bart_summary_docs = [Document(page_content=summary) for summary in bart_summaries]
    
    # Create refine chain
    refine_chain = load_summarize_chain(
        llm=llm,
        chain_type="refine",
        question_prompt=initial_prompt,
        refine_prompt=refine_template,
        return_intermediate_steps=False,
        verbose=True
    )
    
    # Generate final summary
    result = refine_chain.invoke({"input_documents": bart_summary_docs})
    final_summary = result["output_text"]
    
    print("\n✓ Pipeline complete!")
    
    return {
        "final_summary": final_summary,
        "bart_summaries": bart_summaries,
        "chunk_count": len(chunks)
    }